In [12]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
mapper = {
    'pri_neigh':'neighborhood',
    'nearestTrainStopDist_sb':'nearestTransitStopDist_sb',
    'nearestTrainStopDist_nsb':'nearestTransitStopDist_nsb',
    'neighborho':'neighborhood',
    'nearestBusStopDist_sb':'nearestTransitStopDist_sb',
    'nearestBusStopDist_nsb':'nearestTransitStopDist_nsb'
}

In [13]:
sf = pd.read_csv('sf_ratios_081117.csv')
sf.rename(columns=mapper,inplace=True)
print(sf.shape)
sf.head()

(20, 14)


,neighborhood,nearestParkDist_sb,nearestTransitStopDist_sb,nearestCoffeeDist_sb,index_sb,nearestParkDist_nsb,nearestTransitStopDist_nsb,nearestCoffeeDist_nsb,index_nsb,nsbRatio,area,areaPerSB,areaPerNSB,areaPerCoffee
0,Bayview,0.004877,0.000314,0.002994,1,0.004976,0.001146,0.002023,13,13.000000,0.001296,0.001296,0.000100,0.000093
1,Castro/Upper Market,0.003271,0.000266,0.001060,2,0.002343,0.000503,0.001315,14,7.000000,0.000227,0.000113,0.000016,0.000014
2,Downtown/Civic Center,0.002273,0.000368,0.000949,8,0.001886,0.000401,0.000704,22,2.750000,0.000171,0.000021,0.000008,0.000006
3,Financial District,0.003505,0.000457,0.000713,34,0.004371,0.000531,0.000555,45,1.323529,0.000184,0.000005,0.000004,0.000002
4,Inner Richmond,0.004901,0.000766,0.001720,2,0.003120,0.000603,0.001370,15,7.500000,0.000348,0.000174,0.000023,0.000020


In [15]:
chi = pd.read_csv('chicago_ratios_081117.csv')
chi.rename(columns=mapper,inplace=True)
print(chi.shape)
chi.head()

(37, 14)


,neighborhood,nearestCoffeeDist_sb,nearestParkDist_sb,index_sb,nearestTransitStopDist_sb,nearestCoffeeDist_nsb,nearestParkDist_nsb,index_nsb,nearestTransitStopDist_nsb,nsbRatio,areaPerSB,areaPerNSB,areaPerCoffee,area
0,Albany Park,0.006013,0.039140,2,0.008307,0.006013,0.039068,2.0,0.006201,1.0,0.000270,0.000270,0.000135,0.000540
1,Andersonville,0.004972,0.039809,1,0.009828,0.001469,0.044625,1.0,0.009682,1.0,0.000097,0.000097,0.000048,0.000097
2,Belmont Cragin,0.002069,0.074497,2,0.043445,0.020678,0.044437,2.0,0.028794,1.0,0.000550,0.000550,0.000275,0.001100
3,Beverly,0.023791,0.210110,1,0.000000,0.023791,0.188951,1.0,0.054787,1.0,0.000892,0.000892,0.000446,0.000892
4,Boystown,0.001523,0.017254,1,0.009095,0.002895,0.018635,2.0,0.007175,2.0,0.000034,0.000017,0.000011,0.000034


In [16]:
chi_noNeigh = chi[[ x for x in chi.columns if x if 'neighborhood' != x ]]
sf_noNeigh = sf[[ x for x in sf.columns if x if 'neighborhood' != x ]]

In [17]:
sim = pd.DataFrame(cosine_similarity(chi_noNeigh,sf_noNeigh))
sim.columns = sf['neighborhood']
sim.index = chi['neighborhood']

In [6]:
sim

neighborhood,Bayview,Castro/Upper Market,Downtown/Civic Center,Financial District,Inner Richmond,Inner Sunset,Lakeshore,Marina,Mission,Nob Hill,Noe Valley,North Beach,Outer Sunset,Pacific Heights,Potrero Hill,Presidio Heights,Russian Hill,South of Market,West of Twin Peaks,Western Addition
neighborhood,,,,,,,,,,,,,,,,,,,,
Albany Park,0.742265,0.823834,0.887405,0.941354,0.819049,0.872864,0.841185,0.934613,0.724963,0.780267,0.763051,0.881323,0.752362,0.841183,0.829270,0.780267,0.918626,0.853044,0.952571,0.814807
Andersonville,0.846619,0.841496,0.802174,0.821987,0.837486,0.881887,0.927142,0.893940,0.831861,0.878428,0.864129,0.851603,0.855156,0.927140,0.846040,0.878428,0.881536,0.769439,0.942777,0.833921
Belmont Cragin,0.742143,0.823699,0.887259,0.941199,0.818915,0.872721,0.841056,0.934460,0.724843,0.780140,0.762927,0.881179,0.752239,0.841045,0.829134,0.780139,0.918476,0.852904,0.952417,0.814673
Beverly,0.846071,0.840950,0.801652,0.821451,0.836942,0.881316,0.926555,0.893364,0.831319,0.877860,0.863572,0.851052,0.854603,0.926537,0.845491,0.877861,0.880965,0.768939,0.942171,0.833380
Boystown,0.959514,0.929455,0.813161,0.748288,0.927599,0.945604,0.994128,0.906715,0.951515,0.975336,0.968489,0.899879,0.963965,0.994127,0.931510,0.975335,0.906214,0.798761,0.952573,0.925919
Bucktown,0.685306,0.799621,0.902753,0.969141,0.794629,0.851061,0.789471,0.931269,0.667354,0.724976,0.706960,0.874740,0.695812,0.789470,0.805299,0.724976,0.914192,0.868463,0.936582,0.790206
Chatham,0.846644,0.841521,0.802198,0.822012,0.837510,0.881912,0.927168,0.893965,0.831886,0.878453,0.864153,0.851627,0.855181,0.927167,0.846065,0.878453,0.881562,0.769462,0.942804,0.833945
East Village,0.959519,0.929460,0.813166,0.748292,0.927604,0.945609,0.994133,0.906720,0.951520,0.975340,0.968494,0.899883,0.963970,0.994132,0.931515,0.975340,0.906218,0.798765,0.952577,0.925924
Edgewater,0.992731,0.948726,0.790241,0.676656,0.948238,0.949670,0.998402,0.881475,0.989098,0.998336,0.996233,0.896295,0.994556,0.998402,0.949206,0.998337,0.888168,0.787420,0.923868,0.947754
